# **Modelado: Precio de Cierre**

In [2]:
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
btc = pd.read_csv(r'https://raw.githubusercontent.com/TawnyVTC/Proyectos_UN/refs/heads/main/2025/Deep_Learning/Data/btc_1d_with_volatility_and_lags.csv')
btc['Open time'] = pd.to_datetime(btc['Open time'], format='%Y-%m-%d')


## **Split Temporal y Validación Cruzada**


In [ ]:
#Precio de Cierre

features = ['Close', 'LogReturn', 'Volatility',
            'Close_lag_7', 'Close_lag_14', 'Close_lag_21', 'Close_lag_28']

targets = ['target_t+1','target_t+2','target_t+3',
           'target_t+4','target_t+5','target_t+6','target_t+7']

X = btc[features].values
y = btc[targets].values

dates = btc["Open time"]

# -----------------------
# Construcción del objeto timeSeries
# -----------------------
# Si la función espera un formato [X, y] concatenado:
timeSeries = np.concatenate([X, y], axis=1)


In [5]:
from tsxv.splitTrainValTest import split_train_val_test_groupKFold
from sklearn.preprocessing import StandardScaler
import numpy as np

def _ensure_2d(arr):
    """
    Convierte el array a formato 2D.
    Si llega 1D → (n,1); si llega 3D → (n, timesteps*feats)
    """
    arr = np.asarray(arr)
    if arr.ndim == 1:
        return arr.reshape(-1, 1)
    if arr.ndim == 2:
        return arr
    if arr.ndim == 3:
        n, a, b = arr.shape
        return arr.reshape(n, a * b)
    raise ValueError(f"Array con ndim={arr.ndim} no soportado por esta función.")


def split_and_scale(timeSeries, n_steps_input=7, n_steps_forecast=7, n_steps_jump=1, target_col=0):
    X_list, y_list, Xcv_list, ycv_list, Xtest_list, ytest_list = split_train_val_test_groupKFold(
        timeSeries,
        n_steps_input,
        n_steps_forecast,
        n_steps_jump
    )

    X_train_scaled, X_val_scaled, X_test_scaled = [], [], []
    y_train_scaled, y_val_scaled, y_test_scaled = [], [], []
    scalers_x, scalers_y = [], []

    for fold in range(len(X_list)):
        X_train_raw = _ensure_2d(X_list[fold])
        X_val_raw   = _ensure_2d(Xcv_list[fold])
        X_test_raw  = _ensure_2d(Xtest_list[fold])

        # 👇 Aquí filtramos solo la variable objetivo (columna target)
        y_train_raw = _ensure_2d(y_list[fold])[:, target_col:target_col + n_steps_forecast]
        y_val_raw   = _ensure_2d(ycv_list[fold])[:, target_col:target_col + n_steps_forecast]
        y_test_raw  = _ensure_2d(ytest_list[fold])[:, target_col:target_col + n_steps_forecast]

        # Escaladores
        scaler_x = StandardScaler().fit(X_train_raw)
        scaler_y = StandardScaler().fit(y_train_raw)

        X_train_scaled.append(scaler_x.transform(X_train_raw))
        X_val_scaled.append(scaler_x.transform(X_val_raw))
        X_test_scaled.append(scaler_x.transform(X_test_raw))
        y_train_scaled.append(scaler_y.transform(y_train_raw))
        y_val_scaled.append(scaler_y.transform(y_val_raw))
        y_test_scaled.append(scaler_y.transform(y_test_raw))

        scalers_x.append(scaler_x)
        scalers_y.append(scaler_y)

        print(f"Fold {fold+1}: train {X_train_raw.shape} -> val {X_val_raw.shape} -> test {X_test_raw.shape}")

    return {
        'X_train': X_train_scaled,
        'X_val': X_val_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train_scaled,
        'y_val': y_val_scaled,
        'y_test': y_test_scaled,
        'scalers_x': scalers_x,
        'scalers_y': scalers_y
    }


## **Modelado con Deep Learning**

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.stats.diagnostic import acorr_ljungbox
import pandas as pd
import numpy as np

In [7]:
def test_independencia_residuos(residuos, lags=10):
    resultado = acorr_ljungbox(residuos, lags=[lags], return_df=True)
    return resultado['lb_pvalue'].iloc[0]

def _calc_metrics_per_horizon(y_true, y_pred):
    """
    y_true, y_pred: arrays (n_samples, n_horizons)
    Devuelve DataFrame con métricas por horizonte.
    """
    n_outputs = y_true.shape[1]
    rows = []
    for h in range(n_outputs):
        yt = y_true[:, h]
        yp = y_pred[:, h]
        mae = mean_absolute_error(yt, yp)
        mse = mean_squared_error(yt, yp)
        rmse = np.sqrt(mse)
        # MAPE con epsilon para evitar división por cero
        mape = np.mean(np.abs((yt - yp) / (np.abs(yt) + 1e-8))) * 100
        rows.append({'Horizonte': h+1, 'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape})
    df = pd.DataFrame(rows)
    return df

In [8]:
def entrenar_mlp_folds(data_dict, n_outputs=7, epochs=100, batch_size=32, verbose_fit=0):
    """
    Versión extendida:
    - Devuelve: resultados (lista), tablas_folds (lista de df_metrics por fold)
    - Cada df_metrics tiene métricas por horizonte + fila 'Promedio'
    """
    resultados = []
    tablas_folds = []

    for fold in range(len(data_dict['X_train'])):
        print(f"\n===== Fold {fold+1}/{len(data_dict['X_train'])} =====")

        X_train, X_val, X_test = data_dict['X_train'][fold], data_dict['X_val'][fold], data_dict['X_test'][fold]
        y_train, y_val, y_test = data_dict['y_train'][fold], data_dict['y_val'][fold], data_dict['y_test'][fold]
        scaler_y = data_dict['scalers_y'][fold]

        # --- Modelo ---
        model = Sequential([
            Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(n_outputs)
        ])
        model.compile(optimizer='adam', loss='mse')

        es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  epochs=epochs, batch_size=batch_size, verbose=verbose_fit, callbacks=[es])

        # --- Predicciones (desescaladas) ---
        yhat_train = scaler_y.inverse_transform(model.predict(X_train))
        yhat_val = scaler_y.inverse_transform(model.predict(X_val))
        yhat_test = scaler_y.inverse_transform(model.predict(X_test))
        ytrain_real = scaler_y.inverse_transform(y_train)
        yval_real = scaler_y.inverse_transform(y_val)
        ytest_real = scaler_y.inverse_transform(y_test)

        # --- Métricas por horizonte (test) ---
        df_metrics = _calc_metrics_per_horizon(ytest_real, yhat_test)
        # añadir fila promedio
        promedio = df_metrics.mean(numeric_only=True).to_dict()
        promedio['Horizonte'] = 'Promedio'
        df_metrics = pd.concat([df_metrics, pd.DataFrame([promedio])], ignore_index=True)

        # p-value BDS en residuos del horizonte 1 (h=1)
        resid_h1 = ytest_real[:, 0] - yhat_test[:, 0]
        try:
            pval = test_independencia_residuos(resid_h1, lags=10)
        except Exception as e:
            pval = np.nan

        # agregamos columna BDS_pvalue_h1 (solo en la primera fila será pval)
        bds_col = [np.nan] * len(df_metrics)
        if len(df_metrics) > 0:
            bds_col[0] = pval
        df_metrics['BDS_pvalue_h1'] = bds_col

        # print resumen por fold
        print(f"Fold {fold+1} | RMSE prom: {df_metrics.loc[df_metrics['Horizonte']=='Promedio','RMSE'].values[0]:.4f} | pval(h1): {pval}")

        resultados.append({
            'fold': fold+1,
            'rmse_test_prom': df_metrics.loc[df_metrics['Horizonte']=='Promedio','RMSE'].values[0],
            'pval_h1': pval,
            'y_train_real': ytrain_real, 'y_train_pred': yhat_train,
            'y_val_real': yval_real, 'y_val_pred': yhat_val,
            'y_test_real': ytest_real, 'y_test_pred': yhat_test,
            'df_metrics': df_metrics
        })

        tablas_folds.append(df_metrics)

    return resultados, tablas_folds

In [9]:
# ------------------------------------------------------------------
# Funciones para agregar resultados por lag y crear tablas resumen
# ------------------------------------------------------------------

def resumen_por_lag(tablas_folds):
    """
    tablas_folds: lista de dataframes (uno por fold), cada df tiene las filas por horizonte + 'Promedio'.
    Devuelve df resumen por horizonte con mean y std entre folds.
    """
    # extraer solo filas de horizonte 1..H (no 'Promedio') para hacer stats por horizonte
    # asumimos que todos los df tienen la misma cantidad de horizontes antes de la fila Promedio
    list_horizon_dfs = []
    for df in tablas_folds:
        # seleccionar solo filas cuyo 'Horizonte' no sea 'Promedio'
        df_h = df[df['Horizonte'] != 'Promedio'].copy()
        df_h = df_h.reset_index(drop=True)
        list_horizon_dfs.append(df_h)

    # concatenar en multiindex: fold, horizonte
    concat = pd.concat(list_horizon_dfs, keys=range(1, len(list_horizon_dfs)+1), names=['fold','row'])
    # Queremos agrupar por horizonte y calcular mean/std de métricas
    metrics = ['MAPE','MAE','RMSE','MSE']
    summary_rows = []
    for h in sorted(concat['Horizonte'].unique(), key=lambda x: int(x)):
        sel = concat[concat['Horizonte']==h]
        row = {'Horizonte': int(h)}
        for m in metrics:
            row[f'{m}_mean'] = sel[m].mean()
            row[f'{m}_std'] = sel[m].std()
        # p-value mean across folds: buscar pval en fold df (está en la primera fila BDS_pvalue_h1)
        # coletar pvals
        pvals = []
        for df in tablas_folds:
            pv = df['BDS_pvalue_h1'].iloc[0]
            if not np.isnan(pv):
                pvals.append(pv)
        row['BDS_pvalue_h1_mean'] = np.mean(pvals) if len(pvals)>0 else np.nan
        row['BDS_pvalue_h1_std'] = np.std(pvals) if len(pvals)>0 else np.nan

        summary_rows.append(row)

    df_summary = pd.DataFrame(summary_rows)
    return df_summary


In [10]:
# ------------------------------------------------------------------
# Función principal: evalúa lista de lags y devuelve todo lo necesario
# ------------------------------------------------------------------

def evaluar_y_reportar(timeSeries, lags_list=[7,14,21,28], n_outputs=7,
                        epochs=80, batch_size=32, verbose_fit=0):
    """
    Ejecuta todo el flujo para cada n_lag:
    - split + scale
    - entrenar por folds
    - guarda tablas por fold y resumen por lag (mean + std)
    - devuelve estructura con todo para plotting y tablas
    """
    todos_lags = {}
    comparativa_resumen = []

    for n_lag in lags_list:
        print(f"\n=== Procesando n_lag = {n_lag} ===")
        data_dict = split_and_scale(timeSeries, n_steps_input=n_lag, n_steps_forecast=n_outputs)
        resultados, tablas_folds = entrenar_mlp_folds(data_dict, n_outputs=n_outputs,
                                                     epochs=epochs, batch_size=batch_size,
                                                     verbose_fit=verbose_fit)

        # df concatenado de métricas (todos los folds)
        df_metricas_comb = pd.concat([t.reset_index(drop=True) for t in tablas_folds], keys=range(1, len(tablas_folds)+1),
                                      names=['fold','row']).reset_index()
        # resumen por horizonte (mean, std)
        df_summary_h = resumen_por_lag(tablas_folds)

        # resumen global (media de RMSE, MAE, MAPE, MSE y pval promedio)
        rmse_prom = df_metricas_comb.groupby('Horizonte')['RMSE'].mean().mean()  # promedio de horizontes
        mae_prom  = df_metricas_comb.groupby('Horizonte')['MAE'].mean().mean()
        mape_prom = df_metricas_comb.groupby('Horizonte')['MAPE'].mean().mean()
        mse_prom  = df_metricas_comb.groupby('Horizonte')['MSE'].mean().mean()
        pvals = df_metricas_comb['BDS_pvalue_h1'].dropna()
        pval_prom = pvals.mean() if len(pvals)>0 else np.nan

        comparativa_resumen.append({
            'n_lag': n_lag,
            'RMSE_prom': rmse_prom,
            'RMSE_std': df_metricas_comb.groupby('Horizonte')['RMSE'].mean().std(),
            'MAE_prom': mae_prom,
            'MAPE_prom': mape_prom,
            'MSE_prom': mse_prom,
            'BDS_pvalue_h1_prom': pval_prom
        })

        todos_lags[n_lag] = {
            'resultados': resultados,         # lista de dict por fold (incluye df_metrics por fold)
            'tablas_folds': tablas_folds,     # lista de dataframes por fold
            'df_metricas_comb': df_metricas_comb,
            'df_summary_h': df_summary_h
        }

    df_comparativa = pd.DataFrame(comparativa_resumen).sort_values('n_lag').reset_index(drop=True)
    return todos_lags, df_comparativa

In [34]:
# ------------------------------------------------------------------
# Funciones de plotting
# ------------------------------------------------------------------

def escoger_folds_por_rmse(resultados):
    """
    Recibe lista de resultados (cada uno con 'rmse_test_prom').
    Devuelve indices best (min rmse), worst (max rmse) y median (por valor).
    """
    rmses = [r['rmse_test_prom'] for r in resultados]
    order = np.argsort(rmses)
    best_idx = int(order[0])
    worst_idx = int(order[-1])
    median_pos = int(len(order)//2)
    median_idx = int(order[median_pos])
    return best_idx, median_idx, worst_idx


def plot_series_fold(resultados_fold, fold_idx, n_steps_input, title_prefix=""):
    """
    Grafica las series reales y predichas para un fold específico.
    """
    y_train_real = resultados_fold['y_train_real']
    y_train_pred = resultados_fold['y_train_pred']
    y_val_real   = resultados_fold['y_val_real']
    y_val_pred   = resultados_fold['y_val_pred']
    y_test_real  = resultados_fold['y_test_real']
    y_test_pred  = resultados_fold['y_test_pred']

    fig, ax = plt.subplots(figsize=(10, 5))

    # Entrenamiento
    ax.plot(y_train_real[:, 0], label='Train Real', color='tab:blue', alpha=0.6)
    ax.plot(y_train_pred[:, 0], label='Train Predicho', color='tab:cyan', linestyle='--', alpha=0.8)

    # Validación
    ax.plot(range(len(y_train_real), len(y_train_real)+len(y_val_real)),
             y_val_real[:, 0], label='Val Real', color='tab:orange', alpha=0.6)
    ax.plot(range(len(y_train_real), len(y_train_real)+len(y_val_pred)),
             y_val_pred[:, 0], label='Val Predicho', color='tab:red', linestyle='--', alpha=0.8)

    # Test
    ax.plot(range(len(y_train_real)+len(y_val_real),
                  len(y_train_real)+len(y_val_real)+len(y_test_real)),
             y_test_real[:, 0], label='Test Real', color='tab:green', alpha=0.6)
    ax.plot(range(len(y_train_real)+len(y_val_real),
                  len(y_train_real)+len(y_val_real)+len(y_test_pred)),
             y_test_pred[:, 0], label='Test Predicho', color='tab:purple', linestyle='--', alpha=0.8)

    ax.set_title(f"{title_prefix} | Fold {fold_idx+1} | Ventana = {n_steps_input}")
    ax.set_xlabel("Tiempo")
    ax.set_ylabel("Valor (Volatilidad o Precio)")
    ax.legend()
    ax.grid(alpha=0.3)

    fig.tight_layout()
    return fig


def plot_rmse_bars(resultados, n_steps_input):
    """
    Grafica barras con el RMSE promedio por fold.
    """
    import numpy as np
    import matplotlib.pyplot as plt

    rmses = [r['rmse_test_prom'] for r in resultados]
    folds = np.arange(1, len(rmses)+1)

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(folds, rmses, color='skyblue', edgecolor='k')
    ax.set_title(f"RMSE por Fold | Ventana = {n_steps_input}")
    ax.set_xlabel("Fold")
    ax.set_ylabel("RMSE promedio (test)")
    ax.grid(axis='y', alpha=0.3)

    for i, v in enumerate(rmses):
        ax.text(folds[i], v, f"{v:.3f}", ha='center', va='bottom', fontsize=8)

    fig.tight_layout()
    return fig


def plot_rmse_promedio_por_horizonte(df_summary_h, n_steps_input):
    """
    Muestra el RMSE promedio (y std si existe) por horizonte.
    """
    fig, ax = plt.subplots(figsize=(8, 4))

    ax.plot(df_summary_h.index, df_summary_h['RMSE_mean'], marker='o', label='RMSE promedio')

    if 'RMSE_std' in df_summary_h.columns:
        ax.fill_between(df_summary_h.index,
                        df_summary_h['RMSE_mean'] - df_summary_h['RMSE_std'],
                        df_summary_h['RMSE_mean'] + df_summary_h['RMSE_std'],
                        color='blue', alpha=0.2, label='Desviación estándar')

    ax.set_title(f"RMSE promedio por horizonte | Ventana = {n_steps_input}")
    ax.set_xlabel("Horizonte de predicción (h)")
    ax.set_ylabel("RMSE")
    ax.legend()
    ax.grid(alpha=0.3)

    fig.tight_layout()
    return fig


In [12]:
# 1) Ejecutar evaluación completa
todos_lags, df_comparativa = evaluar_y_reportar(timeSeries, lags_list=[7,14,21,28],
                                                n_outputs=7, epochs=80, batch_size=32,
                                                verbose_fit=0)


=== Procesando n_lag = 7 ===
Fold 1: train (331, 98) -> val (109, 98) -> test (109, 98)
Fold 2: train (330, 98) -> val (109, 98) -> test (109, 98)
Fold 3: train (329, 98) -> val (110, 98) -> test (109, 98)
Fold 4: train (328, 98) -> val (110, 98) -> test (109, 98)
Fold 5: train (327, 98) -> val (110, 98) -> test (110, 98)

===== Fold 1/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fold 1 | RMSE prom: 1745.5384 | pval(h1): 0.8967076193728124

===== Fold 2/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Fold 2 | RMSE prom: 1771.4336 | pval(h1): 0.8420617783913587

===== Fold 3/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Fold 3 | RMSE prom: 2222.6178 | pval(h1): 0.12161391817407531

===== Fold 4/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
4/4 ━━━━━━━━━━━━━

### *Tabla de Resumen Comparativo por Lags*

In [13]:
# 2) Ver tabla resumen comparativa de los 4 lags:
df_comparativa.to_csv(r'C:\DeepLearning\DL_Proyecto_2\data\precio_cierre\tablas\resumen_comparativo.csv', index=False)
df_comparativa

,n_lag,RMSE_prom,RMSE_std,MAE_prom,MAPE_prom,MSE_prom,BDS_pvalue_h1_prom
0,7,2208.282385,1413.423755,1615.290503,55.326414,7.168180e+06,0.451281
1,14,3292.791440,2094.215359,2370.556816,90.512134,1.560422e+07,0.382059
2,21,3516.872377,2243.514903,2563.970619,48.764169,1.781065e+07,0.494153
3,28,3786.365659,2434.557203,2851.605577,60.246984,2.084163e+07,0.416129


In [35]:
import os

def resultados_lag_precio(n_lag):
    # 3) Para un lag específico:
    info = todos_lags[n_lag]
    resultados = info['resultados']   # lista por fold
    tablas_folds = info['tablas_folds']
    df_metricas_comb = info['df_metricas_comb']
    df_summary_h = info['df_summary_h']

    # Crear carpetas si no existen
    base_dir = fr'C:\DeepLearning\DL_Proyecto_2\data\precio_cierre'
    tablas_dir = os.path.join(base_dir, 'tablas', f'lag_{n_lag}')
    figs_dir = os.path.join(base_dir, 'figs', f'lag_{n_lag}')
    os.makedirs(tablas_dir, exist_ok=True)
    os.makedirs(figs_dir, exist_ok=True)

    # 4) Guardar tablas por fold (test)
    for i, df in enumerate(tablas_folds):
        print(f"\n--- Fold {i+1} ---")
        df_path = os.path.join(tablas_dir, f'fold_{i+1}_lag_{n_lag}.csv')
        df.to_csv(df_path, index=False)
        print(df)

    # 5) Seleccionar best/median/worst y plotear
    best, med, worst = escoger_folds_por_rmse(resultados)

    # Mejor
    fig = plot_series_fold(resultados[best], best, n_steps_input=n_lag, title_prefix="Mejor (RMSE)")
    fig.savefig(os.path.join(figs_dir, f'mejor_rmse_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close()

    # Mediano
    fig = plot_series_fold(resultados[med], med, n_steps_input=n_lag, title_prefix="Mediano (RMSE)")
    fig.savefig(os.path.join(figs_dir, f'mediano_rmse_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    # Peor
    fig = plot_series_fold(resultados[worst], worst, n_steps_input=n_lag, title_prefix="Peor (RMSE)")
    fig.savefig(os.path.join(figs_dir, f'peor_rmse_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    # 6) Gráficas adicionales
    fig = plot_rmse_bars(resultados, n_steps_input=n_lag)
    fig.savefig(os.path.join(figs_dir, f'rmse_por_fold_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    fig = plot_rmse_promedio_por_horizonte(df_summary_h, n_steps_input=n_lag)
    fig.savefig(os.path.join(figs_dir, f'rmse_promedio_por_horizonte_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    print(f"\n✅ Resultados y gráficas guardados en: {figs_dir}")


### *Tablas de Metricas: Resumen por Horizonte y por Fold*

In [36]:
resultados_lag_precio(7)


--- Fold 1 ---
  Horizonte          MAE           MSE         RMSE        MAPE  BDS_pvalue_h1
0         1  1735.026344  4.911803e+06  2216.258861    9.705390       0.896708
1         2     0.021002  9.858301e-04     0.031398  370.173970            NaN
2         3     0.040650  3.104367e-03     0.055717    7.039458            NaN
3         4  1635.433658  5.398253e+06  2323.413991    8.510498            NaN
4         5  1651.235078  4.988690e+06  2233.537615   11.084910            NaN
5         6  1796.076522  7.228901e+06  2688.661617    9.775077            NaN
6         7  2035.672875  7.599997e+06  2756.809284   10.430466            NaN
7  Promedio  1264.786590  4.303949e+06  1745.538355   60.959967            NaN

--- Fold 2 ---
  Horizonte          MAE           MSE         RMSE        MAPE  BDS_pvalue_h1
0         1  1895.433750  6.203154e+06  2490.613079   10.754533       0.842062
1         2     0.026320  1.334892e-03     0.036536  390.335073            NaN
2         3     0.05

In [37]:
resultados_lag_precio(14)


--- Fold 1 ---
  Horizonte          MAE           MSE         RMSE        MAPE  BDS_pvalue_h1
0         1  3233.590346  1.723555e+07  4151.572035   20.025591       0.043771
1         2     0.022503  1.076387e-03     0.032808  146.455653            NaN
2         3     0.077558  1.675861e-02     0.129455   13.033652            NaN
3         4  3390.903816  2.107813e+07  4591.091995   23.069938            NaN
4         5  3150.933792  1.967249e+07  4435.367854   19.732070            NaN
5         6  3849.734242  2.680600e+07  5177.451179   23.667920            NaN
6         7  3173.518504  3.569445e+07  5974.483407   21.577168            NaN
7  Promedio  2399.825823  1.721237e+07  3475.732676   38.223142            NaN

--- Fold 2 ---
  Horizonte          MAE           MSE         RMSE        MAPE  BDS_pvalue_h1
0         1  4491.185360  3.815902e+07  6177.299105   29.043405       0.468867
1         2     0.027204  1.778403e-03     0.042171  433.888710            NaN
2         3     0.11

In [38]:
resultados_lag_precio(21)


--- Fold 1 ---
  Horizonte          MAE           MSE         RMSE        MAPE  BDS_pvalue_h1
0         1  3861.084526  3.754799e+07  6127.641386   35.550880       0.008995
1         2     0.021012  8.379067e-04     0.028947  268.188922            NaN
2         3     0.069382  9.064884e-03     0.095210   13.258544            NaN
3         4  3257.320415  1.892146e+07  4349.881013   25.145508            NaN
4         5  4006.284282  2.835743e+07  5325.169532   24.280615            NaN
5         6  3412.364703  2.192060e+07  4681.944541   23.429817            NaN
6         7  3868.308455  2.718781e+07  5214.193304   28.308608            NaN
7  Promedio  2629.350397  1.913361e+07  3671.279133   59.737556            NaN

--- Fold 2 ---
  Horizonte          MAE           MSE         RMSE        MAPE  BDS_pvalue_h1
0         1  3755.639323  2.361588e+07  4859.616820   23.795991       0.715309
1         2     0.015532  5.293936e-04     0.023009  200.248563            NaN
2         3     0.07

In [39]:
resultados_lag_precio(28)


--- Fold 1 ---
  Horizonte          MAE           MSE         RMSE        MAPE  BDS_pvalue_h1
0         1  4006.760574  2.677602e+07  5174.555304   28.444437       0.745795
1         2     0.018929  7.860383e-04     0.028036  159.265334            NaN
2         3     0.094704  2.221111e-02     0.149034   14.684018            NaN
3         4  4429.451739  2.886009e+07  5372.159124   25.109386            NaN
4         5  3571.949683  2.533677e+07  5033.564406   24.687273            NaN
5         6  2292.871118  1.248970e+07  3534.077093   16.899919            NaN
6         7  4297.803273  2.715621e+07  5211.161945   23.309130            NaN
7  Promedio  2656.992860  1.723126e+07  3475.099277   41.771357            NaN

--- Fold 2 ---
  Horizonte          MAE           MSE         RMSE        MAPE  BDS_pvalue_h1
0         1  3229.117176  1.575057e+07  3968.699351   20.749108       0.083841
1         2     0.026233  1.042396e-03     0.032286  234.019809            NaN
2         3     0.09